In [1]:
import math
import mysql.connector as mysql
import numpy as np
import os
import pandas as pd
import urllib.request

from os.path import splitext
from pathlib import Path
from PIL import ImageFile

In [2]:
# GENERAL CONFIGS
ImageFile.LOAD_TRUNCATED_IMAGES = True

# == Database config ==
conn = mysql.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor()

# == Files config ==
FILE_FOLDER = r"D:\legal_data\flipkart_com-ecommerce_sample.csv"
fdata=pd.read_csv(Path(FILE_FOLDER))
IMG_FOLDER_DEST = r"D:\images\flipkart"

DB_NAME = "blp_dataset"
TABLE_NAME = "products_tmp"

In [3]:
# Creating environment
cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cursor.execute("USE {}".format(DB_NAME))

cursor.execute("DROP TABLE IF EXISTS products_tmp")

In [4]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS products_tmp ("
    "  id INT(11) AUTO_INCREMENT PRIMARY KEY NOT NULL,"
    "  name TEXT NOT NULL,"
    "  description TEXT,"
    "  price VARCHAR(50),"
    "  market_name VARCHAR(255),"
    "  seller_name TEXT,"
    "  has_image VARCHAR(25) NOT NULL DEFAULT 'has_image',"
    "  illegal ENUM('t', 'f') NOT NULL DEFAULT 't'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [5]:
fdata.columns

Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications'],
      dtype='object')

In [6]:
# Deleting unwanted columns
del_list = ['uniq_id', 'crawl_timestamp', 'product_category_tree', 'pid', 'discounted_price', 
            'is_FK_Advantage_product', 'overall_rating', 'product_specifications']

fdata = fdata.drop(del_list, axis = 1)

In [7]:
def insert_product(row):
    query = "INSERT INTO products_tmp (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
    values = (row['product_name'], row['description'], "Flipkart", row['brand'], "f")

    cursor.execute(query, values)
    conn.commit()

    return cursor.lastrowid

In [8]:
def update_image(product_id, value):
    query = "UPDATE products_tmp SET has_image = %s WHERE id = %s"
    cursor.execute(query, (value, product_id))
    conn.commit()

In [9]:
def get_ext(url):
    filename, file_extension = os.path.splitext(url)
    
    return file_extension

In [10]:
for index, row in fdata.iterrows():
    img_list = row['image']
    
    if row['product_name'] == "NaN" or type(row['product_name']) is not str: row['product_name'] = ""
    if row['description'] == "NaN" or type(row['description']) is not str: row['description'] = ""
    if row['brand'] == "NaN" or type(row['brand']) is not str: row['brand'] = ""
        
    if row['product_name'] is not "" and row['description'] is not "":
        # Insert product into database
        product_id = insert_product(row)

        if img_list and img_list is not None and type(row['product_name']) is str:
            try:
                img_list = img_list.strip('][').split(', ')        
                img_index = 0

                for img in img_list:
                    try:
                        update_image(product_id, "no_image")
                        img = img.strip('\'"')
                        folder_dest = Path(f"{IMG_FOLDER_DEST}\{product_id}{get_ext(img)}")

                        if img_index > 0:
                            folder_dest = Path(f"{IMG_FOLDER_DEST}\{product_id}_{img_index}{get_ext(img)}")

                        try:
                            file = urllib.request.urlretrieve(img, folder_dest)
                            img_index = img_index + 1
                        except:
                            print(f"Product {product_id}: Error retrieving image ({img})")

                        if img_index == 0 : update_image(product_id, "no_image")
                    except:
                        print("First float problem")
                        print(img)
            except:
                print("Float problem")
                print(img)

                update_image(product_id, "no_image")

Product 32: Error retrieving image (http://img5a.flixcart.com/image/fabric/h/k/a/r-c-lehe-bt-indcrown-1000x1000-imaejbczsqzjrbfd.jpeg)
Product 32: Error retrieving image (http://img5a.flixcart.com/image/fabric/h/k/a/r-c-lehe-bt-indcrown-original-imaejbczsqzjrbfd.jpeg)
Product 32: Error retrieving image (http://img6a.flixcart.com/image/fabric/h/k/a/r-c-lehe-bt-indcrown-original-imaejbczsqwbk8yz.jpeg)
Product 32: Error retrieving image (http://img5a.flixcart.com/image/fabric/h/k/a/r-c-lehe-bt-indcrown-original-imaejbczy6tcpa3x.jpeg)
Product 91: Error retrieving image (http://img6a.flixcart.com/image/watch/v/d/5/g-729-s-bk-aries-gold-original-imaecefhwnr3vkgm.jpeg)
Product 91: Error retrieving image (http://img5a.flixcart.com/image/watch/v/d/5/g-729-s-bk-aries-gold-original-imaecefhrbpzq8y9.jpeg)
Product 91: Error retrieving image (http://img5a.flixcart.com/image/watch/v/d/5/g-729-s-bk-aries-gold-original-imaecefhrncfgcqb.jpeg)
Product 91: Error retrieving image (http://img6a.flixcart.com

Product 127: Error retrieving image (http://img6a.flixcart.com/image/watch/g/s/f/dmf-002-gr01-fluid-original-imae5wfnwq6wfyp8.jpeg)
Product 127: Error retrieving image (http://img5a.flixcart.com/image/watch/g/s/f/dmf-002-gr01-fluid-original-imae5wfnwhzermfh.jpeg)
Product 127: Error retrieving image (http://img5a.flixcart.com/image/watch/g/s/f/dmf-002-gr01-fluid-original-imae5wfn3nggjbzc.jpeg)
Product 131: Error retrieving image (http://img6a.flixcart.com/image/jean/q/c/w/626440-wrangler-32-original-imae9hdhvbvhwzeh.jpeg)
Product 131: Error retrieving image (http://img5a.flixcart.com/image/jean/q/c/w/626440-wrangler-32-original-imae9hdhvbvhwzeh.jpeg)
Product 131: Error retrieving image (http://img6a.flixcart.com/image/jean/q/c/w/626440-wrangler-38-original-imae9hdhfgc8s5gv.jpeg)
Product 131: Error retrieving image (http://img6a.flixcart.com/image/jean/q/c/w/626440-wrangler-32-original-imae9hdhnhynbh3d.jpeg)
Product 131: Error retrieving image (http://img6a.flixcart.com/image/jean/q/c/w/

Product 173: Error retrieving image (http://img5a.flixcart.com/image/watch/f/k/y/iw-005-fk-north-moon-original-imae6fxvmewj3t9j.jpeg)
Product 178: Error retrieving image (http://img5a.flixcart.com/image/watch/u/k/d/ad1031-black-skmei-original-imae9mntnc2ezkxd.jpeg)
Product 178: Error retrieving image (http://img6a.flixcart.com/image/watch/u/k/d/ad1031-black-skmei-original-imae9mntnc2ezkxd.jpeg)
Product 178: Error retrieving image (http://img5a.flixcart.com/image/watch/u/k/d/ad1031-black-skmei-original-imae9wwyud9qeay5.jpeg)
Product 178: Error retrieving image (http://img6a.flixcart.com/image/watch/u/k/d/ad1031-black-skmei-original-imae9wwyzfgxaw95.jpeg)
Product 178: Error retrieving image (http://img5a.flixcart.com/image/watch/u/k/d/ad1031-black-skmei-original-imae9mnu8p7cfprh.jpeg)
Product 178: Error retrieving image (http://img6a.flixcart.com/image/watch/u/k/d/ad1031-black-skmei-original-imae9wwymbz2hbyc.jpeg)
Product 184: Error retrieving image (http://img6a.flixcart.com/image/watch

Product 225: Error retrieving image (http://img6a.flixcart.com/image/watch/h/a/u/dg0939-red-skmei-original-imae9mzyagfnzgww.jpeg)
Product 225: Error retrieving image (http://img5a.flixcart.com/image/watch/h/a/u/dg0939-red-skmei-original-imae9mzyubrctbxu.jpeg)
Product 225: Error retrieving image (http://img6a.flixcart.com/image/watch/h/a/u/dg0939-red-skmei-original-imae9mzyhjnfzdph.jpeg)
Product 225: Error retrieving image (http://img6a.flixcart.com/image/watch/h/a/u/dg0939-red-skmei-original-imae9mzyuv78zhpg.jpeg)
Product 225: Error retrieving image (http://img6a.flixcart.com/image/watch/h/a/u/dg0939-red-skmei-original-imae9mzfgythtysa.jpeg)
Product 234: Error retrieving image (http://img5a.flixcart.com/image/watch/g/4/b/goldjack-jack-klein-400x400-imae9bd5yytxgehf.jpeg)
Product 234: Error retrieving image (http://img6a.flixcart.com/image/watch/g/4/b/goldjack-jack-klein-original-imae9bd5yytxgehf.jpeg)
Product 234: Error retrieving image (http://img5a.flixcart.com/image/watch/g/4/b/gold

Product 288: Error retrieving image (http://img6a.flixcart.com/image/watch/h/j/6/205sm005-swissfire-original-imaeay69h8gyapzt.jpeg)
Product 292: Error retrieving image (http://img5a.flixcart.com/image/watch/c/e/m/lgswatch61003pp-digi-yellow-logues-original-imae7yfbnfmxynqh.jpeg)
Product 292: Error retrieving image (http://img6a.flixcart.com/image/watch/c/e/m/lgswatch61003pp-digi-yellow-logues-original-imae7yfbnfmxynqh.jpeg)
Product 292: Error retrieving image (http://img6a.flixcart.com/image/watch/c/e/m/lgswatch61003pp-digi-yellow-logues-original-imae7f2hmpwzzhyv.jpeg)
Product 292: Error retrieving image (http://img6a.flixcart.com/image/watch/c/e/m/lgswatch61003pp-digi-yellow-logues-original-imae7yfbgwhw5cmb.jpeg)
Product 292: Error retrieving image (http://img5a.flixcart.com/image/watch/c/e/m/lgswatch61003pp-digi-yellow-logues-original-imae7yfb2hxfk2nr.jpeg)
Product 309: Error retrieving image (http://img5a.flixcart.com/image/dress/g/r/f/mks05-candied-roses-masaba-for-magic-fairy-8-9-

...

Product 19724: Error retrieving image (http://img6a.flixcart.com/image/dress/m/b/z/1-1-100114-purple-s-love-from-india-s-original-imae7h97zgn678dv.jpeg)
Product 19724: Error retrieving image (http://img6a.flixcart.com/image/dress/g/g/w/1-1-100114-pink-love-from-india-xl-original-imae7h97tvfsjmr5.jpeg)
Product 19724: Error retrieving image (http://img6a.flixcart.com/image/dress/m/b/z/1-1-100114-purple-s-love-from-india-s-original-imae7h97w9hnfz9a.jpeg)
Product 19725: Error retrieving image (http://img5a.flixcart.com/image/dress/s/x/e/1-1-spd1-l-appel-du-vide-s-original-imaebfxvzdbcxqpg.jpeg)
Product 19725: Error retrieving image (http://img5a.flixcart.com/image/dress/s/x/e/1-1-spd1-l-appel-du-vide-s-original-imaebfxuwhm8vgez.jpeg)
Product 19725: Error retrieving image (http://img5a.flixcart.com/image/dress/s/x/e/1-1-spd1-l-appel-du-vide-s-original-imaebfxuugxwfwby.jpeg)
Product 19725: Error retrieving image (http://img6a.flixcart.com/image/dress/s/x/e/1-1-spd1-l-appel-du-vide-s-original

Product 19736: Error retrieving image (http://img6a.flixcart.com/image/dress/t/v/e/1-1-ss-278-kwardrobe-s-original-imaebn4syzugb5rc.jpeg)
Product 19736: Error retrieving image (http://img6a.flixcart.com/image/dress/t/v/e/1-1-ss-278-kwardrobe-l-original-imaebmtsfjvygd6a.jpeg)
Product 19736: Error retrieving image (http://img5a.flixcart.com/image/dress/t/v/e/1-1-ss-278-kwardrobe-s-original-imaebn4sbgbvhpef.jpeg)
Product 19736: Error retrieving image (http://img6a.flixcart.com/image/dress/t/v/e/1-1-ss-278-kwardrobe-l-original-imaebmtst25yug8u.jpeg)
Product 19736: Error retrieving image (http://img5a.flixcart.com/image/dress/t/v/e/1-1-ss-278-kwardrobe-s-original-imaebn4spjesjt7e.jpeg)
Product 19737: Error retrieving image (http://img6a.flixcart.com/image/dress/r/r/7/1-1-301236208ecru-honey-m-original-imae8fvpgwd3s7yg.jpeg)
Product 19737: Error retrieving image (http://img5a.flixcart.com/image/dress/r/r/7/1-1-301236208ecru-honey-m-original-imae8fvpgwd3s7yg.jpeg)
Product 19737: Error retriev

Product 19750: Error retrieving image (http://img5a.flixcart.com/image/dress/w/t/z/1327-red-hermosear-s-original-imae5vujad8tdsax.jpeg)
Product 19750: Error retrieving image (http://img6a.flixcart.com/image/dress/w/t/z/1327-red-hermosear-l-original-imae5vujzyeddgyd.jpeg)
Product 19750: Error retrieving image (http://img5a.flixcart.com/image/dress/w/t/z/1327-red-hermosear-l-original-imae5vujefrhmurw.jpeg)
Product 19751: Error retrieving image (http://img5a.flixcart.com/image/dress/e/c/j/mwsd766f-my-addiction-l-original-imads7xmskxpk2kh.jpeg)
Product 19751: Error retrieving image (http://img6a.flixcart.com/image/dress/e/c/j/mwsd766f-my-addiction-l-original-imads7xmqf6pnn4b.jpeg)
Product 19751: Error retrieving image (http://img6a.flixcart.com/image/dress/e/c/j/mwsd766f-my-addiction-l-original-imads7xmbhgtwynd.jpeg)
Product 19752: Error retrieving image (http://img6a.flixcart.com/image/dress/s/e/y/1-1-1508d19183-mayra-xl-original-imaeahgwwvqkafbd.jpeg)
Product 19752: Error retrieving imag

Product 19763: Error retrieving image (http://img6a.flixcart.com/image/dress/q/h/6/1-1-116black-indi-bargain-free-original-imae8ubapeqgh9mf.jpeg)
Product 19763: Error retrieving image (http://img5a.flixcart.com/image/dress/q/h/6/1-1-116black-indi-bargain-free-original-imae8uba9gfcxm5m.jpeg)
Product 19763: Error retrieving image (http://img5a.flixcart.com/image/dress/q/h/6/1-1-116black-indi-bargain-free-original-imae8ubajackfygr.jpeg)
Product 19763: Error retrieving image (http://img5a.flixcart.com/image/dress/q/h/6/1-1-116black-indi-bargain-free-original-imae8ubagm2njfrv.jpeg)
Product 19764: Error retrieving image (http://img5a.flixcart.com/image/dress/g/h/j/lbd455-ladybug-s-original-imadxgpyzxtjpcsz.jpeg)
Product 19764: Error retrieving image (http://img6a.flixcart.com/image/dress/g/h/j/lbd455-ladybug-s-original-imadxgpyzxtjpcsz.jpeg)
Product 19764: Error retrieving image (http://img5a.flixcart.com/image/dress/g/h/j/lbd455-ladybug-s-original-imadxgpyrzzprrxz.jpeg)
Product 19764: Error

Product 19775: Error retrieving image (http://img6a.flixcart.com/image/dress/t/n/h/1-1-3972-purple-her-grace-s-original-imaearh2r53dbqaz.jpeg)
Product 19775: Error retrieving image (http://img5a.flixcart.com/image/dress/t/n/h/1-1-3972-purple-her-grace-s-original-imaearh2phjtezrn.jpeg)
Product 19775: Error retrieving image (http://img6a.flixcart.com/image/dress/t/n/h/1-1-3972-purple-her-grace-s-original-imaearh2xdzvyhyy.jpeg)
Product 19775: Error retrieving image (http://img5a.flixcart.com/image/dress/t/n/h/1-1-3972-purple-her-grace-s-original-imaearh23regjjcz.jpeg)
Product 19775: Error retrieving image (http://img5a.flixcart.com/image/dress/t/n/h/1-1-3972-purple-her-grace-s-original-imaearh2ghbg2vhs.jpeg)
Product 19776: Error retrieving image (http://img6a.flixcart.com/image/dress/q/h/w/1-1-k-1046b-kaxiaa-l-original-imae96rbcauv8j7z.jpeg)
Product 19776: Error retrieving image (http://img6a.flixcart.com/image/dress/q/h/w/1-1-k-1046b-kaxiaa-l-original-imae96rbdy3zszkv.jpeg)
Product 19776